## Assignment 5
The following assignment will focus on analysing the differences between the sexes by age in Ireland.
Author: Loic Soares Bagnoud


### Part 1
#### Preparing the Data

In [13]:
import pandas as pd
import numpy as np

In [14]:
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df = pd.read_csv(url)
df.tail(3)

,STATISTIC,Statistic Label,TLIST(A1),CensusYear,C02199V02655,Sex,C02076V03371,Single Year of Age,C03789V04537,Administrative Counties,UNIT,VALUE
9789,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-149d-13a3-e055-000000000001,Cavan County Council,Number,12
9790,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-14a4-13a3-e055-000000000001,Donegal County Council,Number,31
9791,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-1495-13a3-e055-000000000001,Monaghan County Council,Number,7


Now I can do the code that is the prep for analysis.py  
I just noticed that there were more columns so I need to get their names

In [15]:
headers = df.columns.tolist()
headers

['STATISTIC',
 'Statistic Label',
 'TLIST(A1)',
 'CensusYear',
 'C02199V02655',
 'Sex',
 'C02076V03371',
 'Single Year of Age',
 'C03789V04537',
 'Administrative Counties',
 'UNIT',
 'VALUE']

In [16]:
drop_col_list = ['STATISTIC', 'Statistic Label','TLIST(A1)','CensusYear','C02199V02655','C02076V03371','C03789V04537','UNIT']

df.drop(columns=drop_col_list, inplace=True)

df = df[df["Single Year of Age"] != "All ages"]
df['Single Year of Age'] = df['Single Year of Age'].str.replace('Under 1 year', '0')
df['Single Year of Age'] = df['Single Year of Age'].str.replace('\D', '', regex=True)

df['Single Year of Age']=df['Single Year of Age'].astype('int64')
df['VALUE']=df['VALUE'].astype('int64')

df = df[df["Sex"] != "Both sexes"]
df.info()

print (df.head(3))


<class 'pandas.core.frame.DataFrame'>
Index: 6464 entries, 3296 to 9791
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Sex                      6464 non-null   object
 1   Single Year of Age       6464 non-null   int64 
 2   Administrative Counties  6464 non-null   object
 3   VALUE                    6464 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 252.5+ KB
       Sex  Single Year of Age Administrative Counties  VALUE
3296  Male                   0                 Ireland  29610
3297  Male                   0   Carlow County Council    346
3298  Male                   0     Dublin City Council   3188


In [17]:
df_anal = pd.pivot_table(df, 'VALUE',["Sex",'Single Year of Age'],"Administrative Counties")
print (df_anal.head(3))
# write out the entire file to local machine
df_anal.to_csv("population_for_analysis.csv")

Administrative Counties    Carlow County Council  Cavan County Council  \
Sex    Single Year of Age                                                
Female 0                                   353.0                 501.0   
       1                                   302.0                 477.0   
       2                                   334.0                 520.0   

Administrative Counties    Clare County Council  Cork City Council  \
Sex    Single Year of Age                                            
Female 0                                  691.0             1124.0   
       1                                  704.0             1136.0   
       2                                  744.0             1162.0   

Administrative Counties    Cork County Council  Donegal County Council  \
Sex    Single Year of Age                                                
Female 0                                2055.0                   881.0   
       1                                2045.0          

Reference: 
I had some issues here with an error I was getting. Luckily, ChatGPT helped me understand what the problem was:

 - https://chatgpt.com/share/68fb8980-d6bc-800b-93f3-d0702e0e6ee1

#### Weighted descriptive statistics

In [18]:
df_sex_only = df[['Sex', 'Single Year of Age', 'VALUE']]
df_sex_only

,Sex,Single Year of Age,VALUE
3296,Male,0,29610
3297,Male,0,346
3298,Male,0,3188
3299,Male,0,1269
3300,Male,0,2059
...,...,...,...
9787,Female,100,7
9788,Female,100,9
9789,Female,100,12
9790,Female,100,31


In [19]:

'''
sex_grouped = (df_sex_only.groupby("Sex"))

weighted_mean_result = sex_grouped.apply(lambda g: np.average(g['Single Year of Age'], weights=g['VALUE']))

weighted_mean_by_sex = weighted_mean_result.reset_index(name='Weighted Mean Age')
weighted_mean_by_sex
'''

# Create an empty list to store results
results = []

# Loop through unique sexes

for sex in df_sex_only['Sex'].unique():
    subset = df_sex_only[df_sex_only['Sex'] == sex]
    weighted_mean = np.average(subset['Single Year of Age'], weights=subset['VALUE'])
    results.append({'Sex': sex, 'Weighted Mean Age': weighted_mean})

# Convert results to DataFrame
weighted_mean_by_sex_loop = pd.DataFrame(results)
weighted_mean_by_sex_loop

,Sex,Weighted Mean Age
0,Male,37.739448
1,Female,38.939796


References:

- For the first solution
>
https://www.geeksforgeeks.org/pandas/python-pandas-dataframe-groupby/
https://realpython.com/pandas-reset-index/

- For the second solution
>
https://chatgpt.com/share/68fc9c34-e5c8-800b-90d2-2a16bfc33ee8

#### Calculating the Difference between the sexes by age

In [20]:
sex_difference = df_sex_only.pivot_table(index='Single Year of Age', columns='Sex', values='VALUE',aggfunc='sum')
sex_difference['Difference (Male - Female)'] = sex_difference['Male'] - sex_difference['Female']
sex_difference

Sex,Female,Male,Difference (Male - Female)
Single Year of Age,,,
0,56372,59220,2848
1,55090,57750,2660
2,57948,60472,2524
3,58966,62002,3036
4,59638,63372,3734
...,...,...,...
96,1912,654,-1258
97,1464,434,-1030
98,984,260,-724


References:

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html

### Part 2
#### Creating the variable and age groups

In [21]:
chosen_age = 35

age_group = df_sex_only[
    (df_sex_only['Single Year of Age'] >= chosen_age - 5) &
    (df_sex_only['Single Year of Age'] <= chosen_age + 5)
]

age_group

,Sex,Single Year of Age,VALUE
4256,Male,30,30858
4257,Male,30,367
4258,Male,30,6163
4259,Male,30,1511
4260,Male,30,1888
...,...,...,...
7867,Female,40,556
7868,Female,40,538
7869,Female,40,630
7870,Female,40,1293


#### Sum up the results and calculate the difference

In [ ]:
# Same logic as above, create a dictionary

results_specific_age = []

# Use a for loop to go through the sexes and sum them up. No average needed this time.
for sex in age_group['Sex'].unique():
    subset = age_group[age_group['Sex'] == sex]
    total_pop = subset['VALUE'].sum()
    results_specific_age.append({'Sex': sex, 'Total Population': total_pop})

# Store it in a dataframe.
sex_ages = pd.DataFrame(results_specific_age)

# Same logic but since this is a new dataframe, we have to create a pivot_table with the new values
sex_difference_age = sex_ages.pivot_table(columns='Sex', values='Total Population')

# Step 4: Calculate difference. We use get()
sex_difference_age['Difference (Male - Female)'] = (
    sex_difference_age.get('Male', 0) - sex_difference_age.get('Female', 0)
)

# Just make sure to convert it to an integer for better clarity
sex_difference_age = sex_difference_age.astype(int)

# Print it
sex_difference_age

Sex,Female,Male,Difference (Male - Female)
Total Population,829012,768060,-60952


References: 
>
https://www.w3schools.com/python/pandas/ref_df_get.asp